# 画像データの前準備

In [ ]:
import os
from pathlib import Path
import sys
import numpy as np
import os, os.path
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.utils import np_utils
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# mpl.rcParams['font.family'] = 'AppleGothic'

### 識別ラベルの表示

In [ ]:
data_dir = Path('./bark_data/')
# list(data_dir.glob('**'))
sp_dir=[x for x in data_dir.iterdir() if x.is_dir()]
sp_list=[x.name for x in data_dir.iterdir() if x.is_dir()]
print(sp_list)
print('合計：'+str(len(sp_list))+'種類')

### 読み込む画像の表示①

In [ ]:
res_sz = 360

h_sz=128 ###### horizontal image size @@@@
v_sz=128 ###### vertical image size @@@@

imgn_list = [len([x for x in sp_dir[i].iterdir() if x.name[0] != '.']) for i in range(len(sp_list))]

fig, axes = plt.subplots(10,len(sp_list), figsize=(15,10))

for i in range(len(sp_list)):
    for j in range(10):
        axes[j, i].axis('off')

for i in range(len(sp_list)):
    for j,fl in enumerate([x for x in sp_dir[i].iterdir() if x.name[0] != '.'][0:10]):
        img = Image.open(os.fspath(fl))
        img = img.resize((res_sz,res_sz))
        img_cr = img.crop((int((res_sz-h_sz)/2),int((res_sz-v_sz)/2),
                    int((res_sz-h_sz)/2+h_sz),int((res_sz-h_sz)/2+v_sz)))
        axes[j, i].imshow(img_cr)
plt.show()

### 読み込む画像の表示②

In [ ]:
imgn_list = [len([x for x in sp_dir[i].iterdir() if x.name[0] != '.']) for i in range(len(sp_list))]

fig, axes = plt.subplots(len(sp_list),10, figsize=(15,40))

for i in range(len(sp_list)):
    for j in range(10):
        axes[i, j].axis('off')

for i in range(len(sp_list)):
    for j,fl in enumerate([x for x in sp_dir[i].iterdir() if x.name[0] != '.'][0:10]):
        img = Image.open(os.fspath(fl))
        img = img.resize((res_sz,res_sz))
        img_cr = img.crop((int((res_sz-h_sz)/2),int((res_sz-v_sz)/2),
                    int((res_sz-h_sz)/2+h_sz),int((res_sz-h_sz)/2+v_sz)))
        axes[i, j].imshow(img_cr)
        axes[i, j].set_title(sp_list[i])
plt.show()

In [ ]:
# imgn_list = [len([x for x in sp_dir[i].iterdir() if x.name[0] != '.']) for i in range(len(sp_list))]

# fig, axes = plt.subplots(len(sp_list),max(imgn_list), figsize=(10,40))

# for i in range(len(sp_list)):
#     for j in range(max(imgn_list)):
#         axes[i, j].axis('off')

# for i in range(len(sp_list)):
#     for j,fl in enumerate([x for x in sp_dir[i].iterdir() if x.name[0] != '.']):
#         img = Image.open(os.fspath(fl))
#         axes[i, j].imshow(img)
#         axes[i, j].set_title(sp_list[i])
# plt.show()

## TrainとValidationに分けて、数値配列として保存

In [ ]:
res_sz = 720

h_sz=128 ###### horizontal image size @@@@
v_sz=128 ###### vertical image size @@@@

#
arr = [] # wood image data
wdid = []  # wood id by number
# resizing initial image data and saving in 4 dimensional array ( samplenumber, h-size, v-size, depth)
for i in range(len(sp_list)):
    for j,fl in enumerate([x for x in sp_dir[i].iterdir() if x.name[0] != '.']):
        img = Image.open(os.fspath(fl))
        img = img.resize((res_sz,res_sz))
        img_cr = img.crop((int((res_sz-h_sz)/2),int((res_sz-v_sz)/2),
                    int((res_sz-h_sz)/2+h_sz),int((res_sz-h_sz)/2+v_sz)))
        arr.append(np.array(img_cr))
        wdid.append(i)
    
img_data = np.array(arr)
img_data.shape

In [ ]:
#  generate train and test sets (5:1) -> test_size=0.166
X_train, x_test, Y_train, y_test = train_test_split(img_data,wdid, test_size=0.166, random_state=42)
# normalize inputs from 0-255 and 0.0-1.0
X_train = X_train.astype("float32")
x_test = x_test.astype("float32")
X_train /=255
x_test /=255

# substraction by mean using private function
X_train = X_train-np.mean(X_train)
x_test= x_test-np.mean(x_test)


# save data
np.savez_compressed('./bark_data_np/train_'+str(h_sz)+'_'+ str(v_sz)+'.npz', X_train,Y_train,sp_list)
np.savez_compressed('./bark_data_np/test_'+str(h_sz)+'_'+ str(v_sz)+'.npz' , x_test, y_test, sp_list)
